** 주요 특징:
- 한국어 전용: 한국어의 특성을 고려한 감성사전
- 품사별 분류: 명사, 동사, 형용사, 부사 등 품사별로 감성 단어 분류
- 긍정/부정 레이블: 각 단어에 긍정(1) 또는 부정(0) 레이블 부여
- 학술적 기반: 한국어 자연어처리 연구에서 널리 사용되는 표준 사전

## 1단계 : total_test_tokens.csv 생성 (평점)

In [7]:
# 1단계: 데이터 로드 및 분석
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.svm import SVC, SVR
from sklearn.metrics import classification_report, confusion_matrix, mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
import warnings
warnings.filterwarnings('ignore')

# 데이터 로드
df = pd.read_csv('/Users/Shared/최종선_교수님/Face_skin_disease/데이터 전처리/Ntoken_review.csv', encoding='utf-8')
df['rating'] = pd.to_numeric(df['rating'], errors='coerce')

print(f"데이터 크기: {df.shape}")
print("평점 분포:")
print(df['rating'].value_counts().sort_index())
print("\n평점별 비율:")
print(df['rating'].value_counts(normalize=True).sort_index())


데이터 크기: (1094, 11)
평점 분포:
rating
1      9
2      9
3     42
4    131
5    903
Name: count, dtype: int64

평점별 비율:
rating
1    0.008227
2    0.008227
3    0.038391
4    0.119744
5    0.825411
Name: proportion, dtype: float64


In [8]:
# 기존 사전 로드
with open('pos_pol_word.txt', 'r', encoding='utf-8') as f:
    pos_words = [line.strip() for line in f.readlines()]

with open('neg_pol_word.txt', 'r', encoding='utf-8') as f:
    neg_words = [line.strip() for line in f.readlines()]

print(f"기존 긍정 사전: {len(pos_words)}개")
print(f"기존 부정 사전: {len(neg_words)}개")

기존 긍정 사전: 4868개
기존 부정 사전: 9827개


### 화장품 도메인 특화 감성사전 생성

In [9]:
# 2단계: 화장품 특화 감성사전 구축
from konlpy.tag import Komoran
from collections import Counter

# 화장품 도메인 특화 감성사전 생성
cosmetic_pos_words = [
    # 보습/수분 관련
    '보습', '수분', '촉촉', '윤기', '탄력', '부드럽', '순하', '진정', '완화',
    '효과', '좋', '만족', '추천', '재구매', '사용감', '발림성', '흡수',
    
    # 긍정 표현
    '최고', '완벽', '대만족', '훌륭', '훌륭하', '훌륭한', '훌륭하다', '훌륭해',
    '좋아', '좋아요', '좋다', '좋은', '좋네', '좋네요', '좋아서', '좋았',
    '만족', '만족하', '만족해', '만족해요', '만족스러', '만족스럽', '만족스러워',
    '추천', '추천하', '추천해', '추천해요', '추천드리', '추천드려', '추천드려요',
    
    # 브랜드/제품 긍정
    '신뢰', '믿음', '믿을', '믿고', '믿어', '믿어요', '믿는다', '믿어',
    '품질', '고품질', '우수', '우수하', '우수한', '우수해', '우수해요',
    
    # 사용 경험 긍정
    '편리', '편리하', '편리한', '편리해', '편리해요', '편리하게',
    '쉽', '쉬워', '쉬워요', '쉬운', '쉽게', '쉽고',
    '빠르', '빠른', '빠르게', '빠르고', '빨라', '빨라요'
]

cosmetic_neg_words = [
    # 부정적 피부 반응
    '자극', '알레르기', '트러블', '여드름', '붉어', '붉은', '붉게', '붉고',
    '가려움', '가려워', '가려워요', '간지러', '간지러워', '간지러워요',
    '부어', '부었', '부어요', '부었어요', '부종', '부으',
    '따갑', '따가워', '따가워요', '따갑고', '따갑게',
    
    # # 제품 문제
    # '끈적', '끈적해', '끈적해요', '끈적하고', '끈적하게', '끈적거리',
    # '무겁', '무거워', '무거워요', '무거운', '무겁게', '무겁고',
    # '기름지', '기름져', '기름져요', '기름진', '기름지게', '기름지고',
    # '번들', '번들거리', '번들거려', '번들거려요', '번들거리고',
    
    # 부정 표현
    '별로', '별로야', '별로예요', '별로네', '별로네요', '별로고',
    '아쉽', '아쉬워', '아쉬워요', '아쉬운', '아쉽게', '아쉽고',
    '실망', '실망하', '실망해', '실망해요', '실망스러', '실망스럽',
    '후회', '후회하', '후회해', '후회해요', '후회돼', '후회돼요',
    
    # 효과 부족
    '효과없', '효과없어', '효과없어요', '효과없고', '효과없게',
    '변화없', '변화없어', '변화없어요', '변화없고', '변화없게',
    '개선없', '개선없어', '개선없어요', '개선없고', '개선없게',
    
    # 가격/가성비 부정
    '비싸', '비싸요', '비싼', '비싸고', '비싸게', '비싸서',
    '아깝', '아까워', '아까워요', '아까운', '아깝게', '아깝고',
    '돈아까', '돈아까워', '돈아까워요', '돈아까운', '돈아깝게'
]

# 사전 저장
with open('cosmetic_pos_words.txt', 'w', encoding='utf-8') as f:
    for word in cosmetic_pos_words:
        f.write(word + '\n')

with open('cosmetic_neg_words.txt', 'w', encoding='utf-8') as f:
    for word in cosmetic_neg_words:
        f.write(word + '\n')

print(f"화장품 특화 긍정 사전: {len(cosmetic_pos_words)}개")
print(f"화장품 특화 부정 사전: {len(cosmetic_neg_words)}개")
print("✅ cosmetic_pos_words.txt, cosmetic_neg_words.txt 저장 완료!")

화장품 특화 긍정 사전: 80개
화장품 특화 부정 사전: 81개
✅ cosmetic_pos_words.txt, cosmetic_neg_words.txt 저장 완료!


In [10]:
# 기존 노트북의 2-3단계를 화장품 특화 사전으로 교체
print("=== 화장품 특화 감성사전 구축 ===")

# 화장품 특화 사전 로드
with open('cosmetic_pos_words.txt', 'r', encoding='utf-8') as f:
    cosmetic_pos_words = [line.strip() for line in f.readlines()]

with open('cosmetic_neg_words.txt', 'r', encoding='utf-8') as f:
    cosmetic_neg_words = [line.strip() for line in f.readlines()]

# Komoran으로 형태소 분석
komoran = Komoran()
tag_list = ['NNG', 'NNP', 'VV', 'VA', 'XR', 'MAG']

# 긍정사전 정제
pos_token = [komoran.pos(token) for token in cosmetic_pos_words]
pos_token_list = [[token[0] for token in tokens if token[1] in tag_list] for tokens in pos_token]
pos_token_list = [tokens for tokens in pos_token_list if len(tokens) > 0]

# 중복 제거
pos_unique_dict = []
for t in pos_token_list:
    if t not in pos_unique_dict:
        pos_unique_dict.append(t)

# 부정사전 정제
neg_token = [komoran.pos(token) for token in cosmetic_neg_words]
neg_token_list = [[token[0] for token in tokens if token[1] in tag_list] for tokens in neg_token]
neg_token_list = [tokens for tokens in neg_token_list if len(tokens) > 0]

neg_unique_dict = []
for t in neg_token_list:
    if t not in neg_unique_dict:
        neg_unique_dict.append(t)

# 1-gram, 2-gram, 3-gram 분류
pos_dict1, pos_dict2, pos_dict3 = [], [], []
for t in pos_unique_dict:
    if len(t) == 1:
        pos_dict1.append(t[0])
    elif len(t) == 2:
        pos_dict2.append(t)
    elif len(t) == 3:
        pos_dict3.append(t)

neg_dict1, neg_dict2, neg_dict3 = [], [], []
for t in neg_unique_dict:
    if len(t) == 1:
        neg_dict1.append(t[0])
    elif len(t) == 2:
        neg_dict2.append(t)
    elif len(t) == 3:
        neg_dict3.append(t)

print(f"화장품 특화 긍정사전: {len(pos_dict1)}개 (1-gram), {len(pos_dict2)}개 (2-gram), {len(pos_dict3)}개 (3-gram)")
print(f"화장품 특화 부정사전: {len(neg_dict1)}개 (1-gram), {len(neg_dict2)}개 (2-gram), {len(neg_dict3)}개 (3-gram)")

=== 화장품 특화 감성사전 구축 ===
화장품 특화 긍정사전: 28개 (1-gram), 7개 (2-gram), 0개 (3-gram)
화장품 특화 부정사전: 18개 (1-gram), 13개 (2-gram), 0개 (3-gram)


In [11]:
# 기존 KNU 사전 + 화장품 특화 사전 결합
def create_combined_sentiment_dictionary():
    """기존 KNU 사전과 화장품 특화 사전을 결합"""
    print("=== 결합된 감성사전 생성 ===")
    
    # 1. 기존 KNU 사전 로드
    with open('pos_pol_word.txt', 'r', encoding='utf-8') as f:
        knu_pos_words = [line.strip() for line in f.readlines()]
    
    with open('neg_pol_word.txt', 'r', encoding='utf-8') as f:
        knu_neg_words = [line.strip() for line in f.readlines()]
    
    # 2. 화장품 특화 사전 로드
    with open('cosmetic_pos_words.txt', 'r', encoding='utf-8') as f:
        cosmetic_pos_words = [line.strip() for line in f.readlines()]
    
    with open('cosmetic_neg_words.txt', 'r', encoding='utf-8') as f:
        cosmetic_neg_words = [line.strip() for line in f.readlines()]
    
    # 3. 사전 결합 (중복 제거)
    combined_pos_words = list(set(knu_pos_words + cosmetic_pos_words))
    combined_neg_words = list(set(knu_neg_words + cosmetic_neg_words))
    
    print(f"기존 KNU 긍정 사전: {len(knu_pos_words)}개")
    print(f"화장품 긍정 사전: {len(cosmetic_pos_words)}개")
    print(f"결합된 긍정 사전: {len(combined_pos_words)}개")
    print(f"  → 추가된 단어: {len(combined_pos_words) - len(knu_pos_words)}개")
    
    print(f"\n기존 KNU 부정 사전: {len(knu_neg_words)}개")
    print(f"화장품 부정 사전: {len(cosmetic_neg_words)}개")
    print(f"결합된 부정 사전: {len(combined_neg_words)}개")
    print(f"  → 추가된 단어: {len(combined_neg_words) - len(knu_neg_words)}개")
    
    # 4. 결합된 사전 저장
    with open('combined_pos_words.txt', 'w', encoding='utf-8') as f:
        for word in combined_pos_words:
            f.write(word + '\n')
    
    with open('combined_neg_words.txt', 'w', encoding='utf-8') as f:
        for word in combined_neg_words:
            f.write(word + '\n')
    
    print(f"\n✅ 결합된 사전 저장 완료!")
    print(f"  - combined_pos_words.txt: {len(combined_pos_words)}개")
    print(f"  - combined_neg_words.txt: {len(combined_neg_words)}개")
    
    return combined_pos_words, combined_neg_words

# 결합된 사전 생성
combined_pos, combined_neg = create_combined_sentiment_dictionary()


=== 결합된 감성사전 생성 ===
기존 KNU 긍정 사전: 4868개
화장품 긍정 사전: 80개
결합된 긍정 사전: 4927개
  → 추가된 단어: 59개

기존 KNU 부정 사전: 9827개
화장품 부정 사전: 81개
결합된 부정 사전: 9901개
  → 추가된 단어: 74개

✅ 결합된 사전 저장 완료!
  - combined_pos_words.txt: 4927개
  - combined_neg_words.txt: 9901개


In [12]:
# # 결합된 사전으로 total_test_tokens3.csv 생성
# def create_total_test_tokens3_with_combined_dict():
#     """결합된 사전을 이용한 total_test_tokens3.csv 생성"""
#     print("=== 결합된 사전으로 total_test_tokens3.csv 생성 ===")
    
#     # 1. 데이터 로드
#     df = pd.read_csv('/Users/Shared/최종선_교수님/Face_skin_disease/데이터 전처리/Ntoken_review.csv', encoding='utf-8')
#     print(f"원본 데이터 크기: {df.shape}")
    
#     # 2. 결합된 사전 로드
#     with open('combined_pos_words.txt', 'r', encoding='utf-8') as f:
#         combined_pos_words = [line.strip() for line in f.readlines()]
    
#     with open('combined_neg_words.txt', 'r', encoding='utf-8') as f:
#         combined_neg_words = [line.strip() for line in f.readlines()]
    
#     print(f"결합된 긍정 사전: {len(combined_pos_words)}개")
#     print(f"결합된 부정 사전: {len(combined_neg_words)}개")
    
#     # 3. Komoran 초기화
#     komoran = Komoran()
#     tag_list = ['NNG', 'NNP', 'VV', 'VA', 'XR', 'MAG']
    
#     # 4. 결합된 사전을 형태소 분석하여 정제
#     def process_dictionary(words, komoran, tag_list):
#         """사전을 형태소 분석하여 정제"""
#         processed_words = []
#         for word in words:
#             try:
#                 pos_result = komoran.pos(word)
#                 filtered_tokens = [token[0] for token in pos_result if token[1] in tag_list]
#                 if filtered_tokens:
#                     processed_words.extend(filtered_tokens)
#             except:
#                 continue
#         return list(set(processed_words))  # 중복 제거
    
#     # 결합된 사전 정제
#     combined_pos_processed = process_dictionary(combined_pos_words, komoran, tag_list)
#     combined_neg_processed = process_dictionary(combined_neg_words, komoran, tag_list)
    
#     print(f"정제된 결합 긍정 사전: {len(combined_pos_processed)}개")
#     print(f"정제된 결합 부정 사전: {len(combined_neg_processed)}개")
    
#     # 5. 리뷰 텍스트를 형태소 분석하고 감성 점수 계산
#     def calculate_sentiment_score(text, pos_words, neg_words, komoran, tag_list):
#         """리뷰 텍스트의 감성 점수 계산"""
#         if pd.isna(text):
#             return 0.0
        
#         try:
#             # 형태소 분석
#             pos_result = komoran.pos(str(text))
#             tokens = [token[0] for token in pos_result if token[1] in tag_list]
            
#             # 감성 단어 카운트
#             pos_count = sum(1 for token in tokens if token in pos_words)
#             neg_count = sum(1 for token in tokens if token in neg_words)
            
#             # 감성 점수 계산 (긍정 단어 - 부정 단어)
#             total_words = len(tokens)
#             if total_words == 0:
#                 return 0.0
            
#             sentiment_score = (pos_count - neg_count) / total_words
#             return sentiment_score
            
#         except Exception as e:
#             print(f"오류 발생: {e}")
#             return 0.0
    
#     # 6. 각 리뷰에 대해 감성 점수 계산
#     print("리뷰 감성 분석 중...")
#     sentiment_scores = []
    
#     for idx, row in df.iterrows():
#         if idx % 100 == 0:
#             print(f"진행률: {idx}/{len(df)} ({idx/len(df)*100:.1f}%)")
        
#         score = calculate_sentiment_score(
#             row['review'], 
#             combined_pos_processed, 
#             combined_neg_processed, 
#             komoran, 
#             tag_list
#         )
#         sentiment_scores.append(score)
    
#     # 7. 결과 데이터프레임 생성
#     result_df = pd.DataFrame({
#         'sen': df['review'].tolist(),
#         'pos_neg': sentiment_scores
#     })
    
#     # 8. 감성 점수 분포 확인
#     print("\n=== 감성 점수 분포 ===")
#     print(f"평균 감성 점수: {np.mean(sentiment_scores):.4f}")
#     print(f"표준편차: {np.std(sentiment_scores):.4f}")
#     print(f"최솟값: {np.min(sentiment_scores):.4f}")
#     print(f"최댓값: {np.max(sentiment_scores):.4f}")
    
#     # 감성 점수별 분포
#     positive_count = sum(1 for score in sentiment_scores if score > 0)
#     negative_count = sum(1 for score in sentiment_scores if score < 0)
#     neutral_count = sum(1 for score in sentiment_scores if score == 0)
    
#     print(f"\n긍정 (점수 > 0): {positive_count}개 ({positive_count/len(sentiment_scores)*100:.1f}%)")
#     print(f"부정 (점수 < 0): {negative_count}개 ({negative_count/len(sentiment_scores)*100:.1f}%)")
#     print(f"중립 (점수 = 0): {neutral_count}개 ({neutral_count/len(sentiment_scores)*100:.1f}%)")
    
#     # 9. CSV 파일 저장
#     output_file = 'total_test_tokens3.csv'
#     result_df.to_csv(output_file, index=False, encoding='utf-8')
#     print(f"\n✅ {output_file} 저장 완료!")
#     print(f"저장된 데이터 크기: {result_df.shape}")
    
#     return result_df

# # 실행
# total_test_tokens3 = create_total_test_tokens3_with_combined_dict()


In [13]:
# 세 가지 사전 성능 비교 분석
def compare_sentiment_dictionaries():
    """세 가지 사전의 성능 비교"""
    print("=== 세 가지 사전 성능 비교 ===")
    
    # 1. 데이터 로드
    df = pd.read_csv('/Users/Shared/최종선_교수님/Face_skin_disease/데이터 전처리/Ntoken_review.csv', encoding='utf-8')
    
    # 2. 실제 감성 레이블 생성 (평점 기반)
    df['actual_sentiment'] = df['rating'].apply(lambda x: 1 if x >= 4 else 0)
    
    # 3. 각 사전별 성능 평가
    results = {}
    
    # 3-1. 기존 KNU 사전
    print("\n1. 기존 KNU 사전 성능:")
    with open('pos_pol_word.txt', 'r', encoding='utf-8') as f:
        knu_pos = [line.strip() for line in f.readlines()]
    with open('neg_pol_word.txt', 'r', encoding='utf-8') as f:
        knu_neg = [line.strip() for line in f.readlines()]
    
    knu_accuracy = evaluate_dictionary_performance(df, knu_pos, knu_neg, "KNU")
    results['KNU'] = knu_accuracy
    
    # 3-2. 화장품 특화 사전
    print("\n2. 화장품 특화 사전 성능:")
    with open('cosmetic_pos_words.txt', 'r', encoding='utf-8') as f:
        cosmetic_pos = [line.strip() for line in f.readlines()]
    with open('cosmetic_neg_words.txt', 'r', encoding='utf-8') as f:
        cosmetic_neg = [line.strip() for line in f.readlines()]
    
    cosmetic_accuracy = evaluate_dictionary_performance(df, cosmetic_pos, cosmetic_neg, "화장품 특화")
    results['화장품 특화'] = cosmetic_accuracy
    
    # 3-3. 결합된 사전
    print("\n3. 결합된 사전 성능:")
    with open('combined_pos_words.txt', 'r', encoding='utf-8') as f:
        combined_pos = [line.strip() for line in f.readlines()]
    with open('combined_neg_words.txt', 'r', encoding='utf-8') as f:
        combined_neg = [line.strip() for line in f.readlines()]
    
    combined_accuracy = evaluate_dictionary_performance(df, combined_pos, combined_neg, "결합된 사전")
    results['결합된 사전'] = combined_accuracy
    
    # 4. 결과 요약
    print("\n" + "="*50)
    print("📊 성능 비교 요약")
    print("="*50)
    
    for name, accuracy in results.items():
        print(f"{name:12}: {accuracy:.3f}")
    
    # 최고 성능 사전 찾기
    best_dict = max(results, key=results.get)
    print(f"\n🏆 최고 성능: {best_dict} ({results[best_dict]:.3f})")
    
    return results

def evaluate_dictionary_performance(df, pos_words, neg_words, dict_name):
    """사전 성능 평가"""
    komoran = Komoran()
    tag_list = ['NNG', 'NNP', 'VV', 'VA', 'XR', 'MAG']
    
    # 사전 정제
    def process_dictionary(words, komoran, tag_list):
        processed_words = []
        for word in words:
            try:
                pos_result = komoran.pos(word)
                filtered_tokens = [token[0] for token in pos_result if token[1] in tag_list]
                if filtered_tokens:
                    processed_words.extend(filtered_tokens)
            except:
                continue
        return list(set(processed_words))
    
    pos_processed = process_dictionary(pos_words, komoran, tag_list)
    neg_processed = process_dictionary(neg_words, komoran, tag_list)
    
    # 감성 분석
    def simple_sentiment_analysis(text, pos_words, neg_words):
        if pd.isna(text):
            return 0
        
        try:
            pos_result = komoran.pos(str(text))
            tokens = [token[0] for token in pos_result if token[1] in tag_list]
            
            pos_count = sum(1 for token in tokens if token in pos_words)
            neg_count = sum(1 for token in tokens if token in neg_words)
            
            if pos_count > neg_count:
                return 1
            elif neg_count > pos_count:
                return 0
            else:
                return 0.5
        except:
            return 0.5
    
    # 예측 수행
    df['predicted'] = df['review'].apply(
        lambda x: simple_sentiment_analysis(x, pos_processed, neg_processed)
    )
    
    # 정확도 계산
    accuracy = (df['predicted'] == df['actual_sentiment']).mean()
    
    print(f"  사전 크기: 긍정 {len(pos_processed)}개, 부정 {len(neg_processed)}개")
    print(f"  정확도: {accuracy:.3f}")
    
    return accuracy

# 성능 비교 실행
comparison_results = compare_sentiment_dictionaries()


=== 세 가지 사전 성능 비교 ===

1. 기존 KNU 사전 성능:
  사전 크기: 긍정 1716개, 부정 2980개
  정확도: 0.425

2. 화장품 특화 사전 성능:
  사전 크기: 긍정 35개, 부정 31개
  정확도: 0.751

3. 결합된 사전 성능:
  사전 크기: 긍정 1727개, 부정 2994개
  정확도: 0.518

📊 성능 비교 요약
KNU         : 0.425
화장품 특화      : 0.751
결합된 사전      : 0.518

🏆 최고 성능: 화장품 특화 (0.751)
